# 🔍 Hyperlane Token Fee Explorer

Interactive and comprehensive visualization of Linear, Progressive, and Regressive fee structures.

**Features:**
- 🎛️ Interactive sliders for real-time parameter exploration
- 📊 Comprehensive 4-panel analysis view
- 💰 USD and token amount display modes
- 💾 SVG export functionality
- 📈 Key transfer amounts: $100, $1,000, $10,000, $100,000 (USDC: 1 USD = 10^6 tokens)

In [ ]:
# Install required packages (uncomment and run if needed)
# !pip install ipywidgets matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown, Layout, HBox, VBox, Button, Output, Checkbox
from IPython.display import display, HTML, clear_output
import io

# Enable interactive plots
%matplotlib inline

## Fee Calculation Functions

In [ ]:
def linear_fee(amount, max_fee, half_amount):
    """Linear Fee: fee = min(maxFee, (amount * maxFee) / (2 * halfAmount))"""
    uncapped = (amount * max_fee) / (2 * half_amount)
    return np.minimum(uncapped, max_fee)

def progressive_fee(amount, max_fee, half_amount):
    """Progressive Fee: fee = (maxFee * amount²) / (halfAmount² + amount²)"""
    if isinstance(amount, np.ndarray):
        return np.where(amount == 0, 0,
                       (max_fee * amount**2) / (half_amount**2 + amount**2))
    return 0 if amount == 0 else (max_fee * amount**2) / (half_amount**2 + amount**2)

def regressive_fee(amount, max_fee, half_amount):
    """Regressive Fee: fee = (maxFee * amount) / (halfAmount + amount)"""
    denominator = half_amount + amount
    return np.where(denominator == 0, 0, (max_fee * amount) / denominator)

## Visualization Functions

In [ ]:
# Key transfer amounts to highlight (USDC: 1 USD = 10^6 tokens)
KEY_AMOUNTS = [100 * 10**6, 1000 * 10**6, 10000 * 10**6, 100000 * 10**6]
KEY_AMOUNT_LABELS = ['$100', '$1K', '$10K', '$100K']
KEY_AMOUNT_COLORS = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

def format_currency(value, pos):
    """Format currency with K/M notation"""
    if value >= 1_000_000:
        return f'${value/1_000_000:.1f}M'
    elif value >= 1_000:
        return f'${value/1_000:.0f}K'
    elif value >= 1:
        return f'${value:.0f}'
    else:
        return f'${value:.2f}'

def format_tokens(value, pos):
    """Format token amounts with K/M notation"""
    if value >= 1_000_000:
        return f'{value/1_000_000:.1f}M'
    elif value >= 1_000:
        return f'{value/1_000:.0f}K'
    else:
        return f'{value:.0f}'

def create_comprehensive_visualization(max_fee=1000, half_amount=10000, max_amount=50000, use_dollars=False):
    """
    Create comprehensive 4-panel fee curve visualization (comprehensive 4-panel analysis).
    """
    amounts = np.linspace(0, max_amount, 1000)
    
    # Calculate fees
    linear_fees = linear_fee(amounts, max_fee, half_amount)
    progressive_fees = progressive_fee(amounts, max_fee, half_amount)
    regressive_fees = regressive_fee(amounts, max_fee, half_amount)
    
    # Calculate percentages
    linear_pct = np.where(amounts > 0, (linear_fees / amounts) * 100, 0)
    progressive_pct = np.where(amounts > 0, (progressive_fees / amounts) * 100, 0)
    regressive_pct = np.where(amounts > 0, (regressive_fees / amounts) * 100, 0)
    
    # Convert to display units
    divisor = 10**6 if use_dollars else 1
    amounts_display = amounts / divisor
    linear_fees_display = linear_fees / divisor
    progressive_fees_display = progressive_fees / divisor
    regressive_fees_display = regressive_fees / divisor
    max_fee_display = max_fee / divisor
    half_amount_display = half_amount / divisor
    max_amount_display = max_amount / divisor
    
    colors = {'linear': '#2E86AB', 'progressive': '#A23B72', 'regressive': '#F18F01'}
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    unit_label = 'USD' if use_dollars else 'tokens'
    fig.suptitle(f'Hyperlane Token Fee Structures Comparison', fontsize=18, fontweight='bold', y=0.995)
    
    from matplotlib.ticker import FuncFormatter
    formatter = FuncFormatter(format_currency if use_dollars else format_tokens)
    
    # ========== Plot 1: Absolute Fees ==========
    ax1 = axes[0, 0]
    ax1.plot(amounts_display, linear_fees_display, label='Linear', linewidth=2.5, color=colors['linear'])
    ax1.plot(amounts_display, progressive_fees_display, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax1.plot(amounts_display, regressive_fees_display, label='Regressive', linewidth=2.5, color=colors['regressive'])
    ax1.axhline(y=max_fee_display, color='red', linestyle='--', alpha=0.4, linewidth=1.5, label=f'maxFee')
    ax1.axvline(x=half_amount_display, color='gray', linestyle='--', alpha=0.3, linewidth=1.5)
    ax1.axhline(y=max_fee_display/2, color='gray', linestyle='--', alpha=0.3, linewidth=1.5)
    ax1.scatter([half_amount_display], [max_fee_display/2], color='red', s=150, zorder=5,
                marker='o', edgecolors='black', linewidths=2, label=f'Intersection')
    
    ax1.set_xlabel(f'Transfer Amount ({unit_label})', fontsize=12, fontweight='bold')
    ax1.set_ylabel(f'Absolute Fee ({unit_label})', fontsize=12, fontweight='bold')
    ax1.set_title('Absolute Fee vs Transfer Amount', fontsize=13, fontweight='bold', pad=10)
    ax1.legend(loc='lower right', fontsize=10, framealpha=0.95)
    ax1.grid(True, alpha=0.3, linestyle=':')
    ax1.set_xlim(0, max_amount_display)
    ax1.set_ylim(0, max_fee_display * 1.1)
    ax1.xaxis.set_major_formatter(formatter)
    ax1.yaxis.set_major_formatter(formatter)
    
    # ========== Plot 2: Fee Percentages ==========
    ax2 = axes[0, 1]
    ax2.plot(amounts_display, linear_pct, label='Linear', linewidth=2.5, color=colors['linear'])
    ax2.plot(amounts_display, progressive_pct, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax2.plot(amounts_display, regressive_pct, label='Regressive', linewidth=2.5, color=colors['regressive'])
    ax2.axvline(x=half_amount_display, color='gray', linestyle='--', alpha=0.3, linewidth=1.5, label='halfAmount')
    
    ax2.set_xlabel(f'Transfer Amount ({unit_label})', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Fee Percentage (%)', fontsize=12, fontweight='bold')
    ax2.set_title('Fee Percentage vs Transfer Amount', fontsize=13, fontweight='bold', pad=10)
    ax2.legend(loc='best', fontsize=10, framealpha=0.95)
    ax2.grid(True, alpha=0.3, linestyle=':')
    ax2.set_xlim(0, max_amount_display)
    
    max_pct = max(
        np.max(linear_pct[amounts <= half_amount*2]) if len(linear_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(progressive_pct[amounts <= half_amount*2]) if len(progressive_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(regressive_pct[amounts <= half_amount*2]) if len(regressive_pct[amounts <= half_amount*2]) > 0 else 10
    )
    ax2.set_ylim(0, min(20, max_pct * 1.2))
    ax2.xaxis.set_major_formatter(formatter)
    
    # ========== Plot 3: Zoomed View ==========
    ax3 = axes[1, 0]
    zoom_mask = (amounts >= half_amount * 0.1) & (amounts <= half_amount * 2.5)
    ax3.plot(amounts_display[zoom_mask], linear_fees_display[zoom_mask], label='Linear',
             linewidth=2.5, color=colors['linear'])
    ax3.plot(amounts_display[zoom_mask], progressive_fees_display[zoom_mask], label='Progressive',
             linewidth=2.5, color=colors['progressive'])
    ax3.plot(amounts_display[zoom_mask], regressive_fees_display[zoom_mask], label='Regressive',
             linewidth=2.5, color=colors['regressive'])
    ax3.axhline(y=max_fee_display/2, color='red', linestyle='--', alpha=0.4, linewidth=1.5, label='maxFee/2')
    ax3.axvline(x=half_amount_display, color='gray', linestyle='--', alpha=0.5, linewidth=1.5, label='halfAmount')
    ax3.scatter([half_amount_display], [max_fee_display/2], color='red', s=150, zorder=5,
                marker='o', edgecolors='black', linewidths=2)
    
    ax3.set_xlabel(f'Transfer Amount ({unit_label})', fontsize=12, fontweight='bold')
    ax3.set_ylabel(f'Absolute Fee ({unit_label})', fontsize=12, fontweight='bold')
    ax3.set_title(f'Zoomed View: All Curves Intersect at halfAmount',
                  fontsize=13, fontweight='bold', pad=10)
    ax3.legend(loc='best', fontsize=10, framealpha=0.95)
    ax3.grid(True, alpha=0.3, linestyle=':')
    ax3.xaxis.set_major_formatter(formatter)
    ax3.yaxis.set_major_formatter(formatter)
    
    # ========== Plot 4: Comparison Table ==========
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    test_amounts = [
        half_amount * 0.2,
        half_amount * 0.5,
        half_amount,
        half_amount * 2,
        half_amount * 5
    ]
    
    table_data = []
    for amt in test_amounts:
        amt_display = amt / divisor
        lin_fee = float(linear_fee(amt, max_fee, half_amount)) / divisor
        prog_fee = float(progressive_fee(amt, max_fee, half_amount)) / divisor
        reg_fee = float(regressive_fee(amt, max_fee, half_amount)) / divisor
        
        prefix = '$' if use_dollars else ''
        table_data.append([
            f'{prefix}{amt_display:,.0f}',
            f'{prefix}{lin_fee:.1f}',
            f'{lin_fee/amt_display*100:.2f}%' if amt_display > 0 else '0%',
            f'{prefix}{prog_fee:.1f}',
            f'{prog_fee/amt_display*100:.2f}%' if amt_display > 0 else '0%',
            f'{prefix}{reg_fee:.1f}',
            f'{reg_fee/amt_display*100:.2f}%' if amt_display > 0 else '0%'
        ])
    
    table = ax4.table(
        cellText=table_data,
        colLabels=['Amount', 'Linear\nFee', 'Linear\n%', 'Progressive\nFee', 'Progressive\n%', 'Regressive\nFee', 'Regressive\n%'],
        cellLoc='center',
        loc='center',
        bbox=[0, 0.1, 1, 0.8]
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.2)
    
    for i in range(7):
        table[(0, i)].set_facecolor('#D3D3D3')
        table[(0, i)].set_text_props(weight='bold', fontsize=9)
    
    for i in range(1, len(test_amounts) + 1):
        for j in range(7):
            if i % 2 == 0:
                table[(i, j)].set_facecolor('#F5F5F5')
            if test_amounts[i-1] == half_amount:
                table[(i, j)].set_facecolor('#FFE6E6')
    
    ax4.text(0.5, 0.95, 'Fee Comparison at Key Transfer Amounts',
             ha='center', va='top', fontsize=13, fontweight='bold',
             transform=ax4.transAxes)
    
    param_text = f'Parameters: maxFee = {format_currency(max_fee_display, None) if use_dollars else f"{max_fee_display:,.0f} tokens"} | halfAmount = {format_currency(half_amount_display, None) if use_dollars else f"{half_amount_display:,.0f} tokens"}'
    ax4.text(0.5, 0.02, param_text, ha='center', va='bottom',
             fontsize=9, style='italic', transform=ax4.transAxes)
    
    plt.tight_layout()
    return fig

def create_interactive_visualization(max_fee=1000000, half_amount=10000000, selected_amount=10000000):
    """
    Create interactive 2-panel visualization with key amount highlights (interactive 2-panel view with real-time controls).
    """
    max_display = max(100000 * 10**6, max(KEY_AMOUNTS) * 1.2, half_amount * 2)
    amounts = np.linspace(0, max_display, 1000)
    
    linear_fees = linear_fee(amounts, max_fee, half_amount)
    progressive_fees = progressive_fee(amounts, max_fee, half_amount)
    regressive_fees = regressive_fee(amounts, max_fee, half_amount)
    
    with np.errstate(divide='ignore', invalid='ignore'):
        linear_pct = np.where(amounts > 0, (linear_fees / amounts) * 100, 0)
        progressive_pct = np.where(amounts > 0, (progressive_fees / amounts) * 100, 0)
        regressive_pct = np.where(amounts > 0, (regressive_fees / amounts) * 100, 0)
    
    # Convert to USD
    amounts_usd = amounts / 10**6
    linear_fees_usd = linear_fees / 10**6
    progressive_fees_usd = progressive_fees / 10**6
    regressive_fees_usd = regressive_fees / 10**6
    max_fee_usd = max_fee / 10**6
    half_amount_usd = half_amount / 10**6
    selected_amount_usd = selected_amount / 10**6
    
    colors = {'linear': '#2E86AB', 'progressive': '#A23B72', 'regressive': '#F18F01'}
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Hyperlane Token Fee Structures', fontsize=16, fontweight='bold')
    
    from matplotlib.ticker import FuncFormatter
    
    # Plot 1: Absolute Fees
    ax1 = axes[0]
    ax1.plot(amounts_usd, linear_fees_usd, label='Linear', linewidth=2.5, color=colors['linear'])
    ax1.plot(amounts_usd, progressive_fees_usd, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax1.plot(amounts_usd, regressive_fees_usd, label='Regressive', linewidth=2.5, color=colors['regressive'])
    
    # Highlight key amounts
    for i, amt in enumerate(KEY_AMOUNTS):
        amt_usd = amt / 10**6
        if amt <= max_display:
            ax1.axvline(x=amt_usd, color=KEY_AMOUNT_COLORS[i], linestyle=':', alpha=0.4, linewidth=2)
            lin_fee = linear_fee(amt, max_fee, half_amount) / 10**6
            prog_fee = progressive_fee(amt, max_fee, half_amount) / 10**6
            reg_fee = regressive_fee(amt, max_fee, half_amount) / 10**6
            ax1.scatter([amt_usd], [lin_fee], color=colors['linear'], s=80, zorder=5, 
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax1.scatter([amt_usd], [prog_fee], color=colors['progressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax1.scatter([amt_usd], [reg_fee], color=colors['regressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
    
    # Highlight selected amount
    if selected_amount > 0:
        current_linear = linear_fee(selected_amount, max_fee, half_amount) / 10**6
        current_progressive = progressive_fee(selected_amount, max_fee, half_amount) / 10**6
        current_regressive = regressive_fee(selected_amount, max_fee, half_amount) / 10**6
        ax1.axvline(x=selected_amount_usd, color='red', linestyle='--', alpha=0.6, linewidth=2.5, 
                   label=f'Selected: {format_currency(selected_amount_usd, None)}')
        ax1.scatter([selected_amount_usd], [current_linear], color=colors['linear'], s=200, 
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax1.scatter([selected_amount_usd], [current_progressive], color=colors['progressive'], s=200,
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax1.scatter([selected_amount_usd], [current_regressive], color=colors['regressive'], s=200,
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
    
    ax1.axhline(y=max_fee_usd, color='gray', linestyle='--', alpha=0.3, linewidth=1.5, label='maxFee')
    ax1.axvline(x=half_amount_usd, color='gray', linestyle='-.', alpha=0.4, linewidth=1.5, label='halfAmount')
    ax1.set_xlabel('Transfer Amount (USD)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Absolute Fee (USD)', fontsize=12, fontweight='bold')
    ax1.set_title('Absolute Fees', fontsize=13, fontweight='bold')
    ax1.legend(loc='lower right', fontsize=9)
    ax1.grid(True, alpha=0.3, linestyle=':')
    ax1.set_xlim(0, max_display / 10**6)
    ax1.set_ylim(0, max_fee_usd * 1.1)
    ax1.xaxis.set_major_formatter(FuncFormatter(format_currency))
    ax1.yaxis.set_major_formatter(FuncFormatter(format_currency))
    
    # Plot 2: Fee Percentages
    ax2 = axes[1]
    ax2.plot(amounts_usd, linear_pct, label='Linear', linewidth=2.5, color=colors['linear'])
    ax2.plot(amounts_usd, progressive_pct, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax2.plot(amounts_usd, regressive_pct, label='Regressive', linewidth=2.5, color=colors['regressive'])
    
    # Highlight key amounts
    for i, amt in enumerate(KEY_AMOUNTS):
        amt_usd = amt / 10**6
        if amt <= max_display and amt > 0:
            ax2.axvline(x=amt_usd, color=KEY_AMOUNT_COLORS[i], linestyle=':', alpha=0.4, linewidth=2)
            lin_fee = linear_fee(amt, max_fee, half_amount)
            prog_fee = progressive_fee(amt, max_fee, half_amount)
            reg_fee = regressive_fee(amt, max_fee, half_amount)
            ax2.scatter([amt_usd], [lin_fee/amt*100], color=colors['linear'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax2.scatter([amt_usd], [prog_fee/amt*100], color=colors['progressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax2.scatter([amt_usd], [reg_fee/amt*100], color=colors['regressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
    
    # Highlight selected amount
    if selected_amount > 0:
        ax2.axvline(x=selected_amount_usd, color='red', linestyle='--', alpha=0.6, linewidth=2.5,
                   label=f'Selected: {format_currency(selected_amount_usd, None)}')
        current_linear_raw = linear_fee(selected_amount, max_fee, half_amount)
        current_progressive_raw = progressive_fee(selected_amount, max_fee, half_amount)
        current_regressive_raw = regressive_fee(selected_amount, max_fee, half_amount)
        ax2.scatter([selected_amount_usd], [current_linear_raw/selected_amount*100], color=colors['linear'], 
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax2.scatter([selected_amount_usd], [current_progressive_raw/selected_amount*100], color=colors['progressive'],
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax2.scatter([selected_amount_usd], [current_regressive_raw/selected_amount*100], color=colors['regressive'],
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
    
    ax2.axvline(x=half_amount_usd, color='gray', linestyle='-.', alpha=0.4, linewidth=1.5, label='halfAmount')
    ax2.set_xlabel('Transfer Amount (USD)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Fee Percentage (%)', fontsize=12, fontweight='bold')
    ax2.set_title('Fee Percentages', fontsize=13, fontweight='bold')
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3, linestyle=':')
    ax2.set_xlim(0, max_display / 10**6)
    
    max_pct = max(
        np.max(linear_pct[amounts <= half_amount*2]) if len(linear_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(progressive_pct[amounts <= half_amount*2]) if len(progressive_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(regressive_pct[amounts <= half_amount*2]) if len(regressive_pct[amounts <= half_amount*2]) > 0 else 10
    )
    y_limit = max(1, min(25, max_pct * 1.3))
    ax2.set_ylim(0, y_limit)
    ax2.xaxis.set_major_formatter(FuncFormatter(format_currency))
    
    plt.tight_layout()
    plt.show()
    
    display_comparison_table(max_fee, half_amount, selected_amount)

def display_comparison_table(max_fee, half_amount, selected_amount):
    """Display HTML comparison table for key transfer amounts"""
    rows_html = ""
    for i, amt in enumerate(KEY_AMOUNTS):
        lin_fee = linear_fee(amt, max_fee, half_amount)
        prog_fee = progressive_fee(amt, max_fee, half_amount)
        reg_fee = regressive_fee(amt, max_fee, half_amount)
        
        lin_pct = (lin_fee / amt) * 100 if amt > 0 else 0
        prog_pct = (prog_fee / amt) * 100 if amt > 0 else 0
        reg_pct = (reg_fee / amt) * 100 if amt > 0 else 0
        
        row_style = 'background-color: #fff3cd; font-weight: bold;' if amt == selected_amount else ''
        amt_dollars = amt / 10**6
        lin_fee_dollars = lin_fee / 10**6
        prog_fee_dollars = prog_fee / 10**6
        reg_fee_dollars = reg_fee / 10**6
        
        rows_html += f"""
        <tr style="{row_style}">
            <td style="padding: 10px; border: 1px solid #dee2e6; text-align: center;">
                <span style="display: inline-block; width: 12px; height: 12px; background-color: {KEY_AMOUNT_COLORS[i]}; border-radius: 50%; margin-right: 8px;"></span>
                <strong>{KEY_AMOUNT_LABELS[i]}</strong>
            </td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.1);">${lin_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.05);">{lin_pct:.3f}%</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.1);">${prog_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.05);">{prog_pct:.3f}%</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.1);">${reg_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.05);">{reg_pct:.3f}%</td>
        </tr>
        """
    
    html = f"""
    <div style="margin-top: 20px; padding: 15px; background-color: #f8f9fa; border-radius: 8px; border: 1px solid #dee2e6;">
        <h3 style="margin-top: 0; color: #212529;">📊 Key Transfer Amounts Comparison</h3>
        <table style="width: 100%; border-collapse: collapse; margin-top: 10px;">
            <thead>
                <tr style="background-color: #e9ecef; text-align: center;">
                    <th style="padding: 10px; border: 1px solid #dee2e6;">Transfer Amount</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.2);">Linear</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.2);">Progressive</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.2);">Regressive</th>
                </tr>
                <tr style="background-color: #f8f9fa; text-align: center; font-size: 0.9em;">
                    <th style="padding: 8px; border: 1px solid #dee2e6;"></th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                </tr>
            </thead>
            <tbody>
                {rows_html}
            </tbody>
        </table>
        <p style="margin-top: 15px; margin-bottom: 0; color: #6c757d; font-size: 0.9em;">
            <strong>Parameters:</strong> maxFee = ${max_fee/10**6:,.2f} | halfAmount = ${half_amount/10**6:,.2f}
        </p>
        <p style="margin-top: 5px; margin-bottom: 0; color: #6c757d; font-size: 0.85em;">
            ⭐ <strong>Highlighted row</strong> indicates the currently selected transfer amount | 💡 <strong>1 USDC = 10<sup>6</sup> tokens</strong>
        </p>
    </div>
    """
    display(HTML(html))

def export_svg(max_fee, half_amount, max_amount, use_dollars, filename='fee_curves.svg'):
    """Export comprehensive visualization as single-line SVG"""
    fig = create_comprehensive_visualization(max_fee, half_amount, max_amount, use_dollars)
    buffer = io.StringIO()
    plt.savefig(buffer, format='svg', bbox_inches='tight', facecolor='white')
    svg_content = buffer.getvalue()
    buffer.close()
    plt.close(fig)
    
    # Write as single line
    svg_single_line = ' '.join(svg_content.split())
    with open(filename, 'w') as f:
        f.write(svg_single_line)
    print(f'✓ SVG visualization saved: {filename}')

## 🎛️ Interactive Fee Explorer

Use the sliders below to explore how different parameters affect fee calculations in real-time.

In [ ]:
from ipywidgets import interactive_output, Label

slider_layout = Layout(width='95%')
dropdown_layout = Layout(width='95%')

# Create widgets
max_fee_slider = IntSlider(
    value=100 * 10**6,
    min=1 * 10**6,
    max=5000 * 10**6,
    step=10 * 10**6,
    description='Max Fee:',
    style={'description_width': '120px'},
    layout=slider_layout,
    continuous_update=False
)

half_amount_slider = IntSlider(
    value=10000 * 10**6,
    min=100 * 10**6,
    max=100000 * 10**6,
    step=1000 * 10**6,
    description='Half Amount:',
    style={'description_width': '120px'},
    layout=slider_layout,
    continuous_update=False
)

selected_amount_dropdown = Dropdown(
    options=[
        ('$100', 100 * 10**6),
        ('$1,000', 1000 * 10**6),
        ('$10,000', 10000 * 10**6),
        ('$100,000', 100000 * 10**6),
        ('Custom: $5,000', 5000 * 10**6),
        ('Custom: $25,000', 25000 * 10**6),
        ('Custom: $50,000', 50000 * 10**6)
    ],
    value=10000 * 10**6,
    description='Selected Amount:',
    style={'description_width': '120px'},
    layout=dropdown_layout
)

max_fee_label = Label(value=f'Current: ${max_fee_slider.value / 10**6:,.0f}')
half_amount_label = Label(value=f'Current: ${half_amount_slider.value / 10**6:,.0f}')

def update_max_fee_label(change):
    max_fee_label.value = f'Current: ${change.new / 10**6:,.0f}'

def update_half_amount_label(change):
    half_amount_label.value = f'Current: ${change.new / 10**6:,.0f}'

max_fee_slider.observe(update_max_fee_label, names='value')
half_amount_slider.observe(update_half_amount_label, names='value')

output = interactive_output(
    create_interactive_visualization,
    {
        'max_fee': max_fee_slider,
        'half_amount': half_amount_slider,
        'selected_amount': selected_amount_dropdown
    }
)

controls = VBox([
    HBox([max_fee_slider, max_fee_label], layout=Layout(margin='5px 0')),
    HBox([half_amount_slider, half_amount_label], layout=Layout(margin='5px 0')),
    selected_amount_dropdown
], layout=Layout(padding='10px', width='100%'))

display(HTML("""
<div style="background-color: #e3f2fd; padding: 12px; border-radius: 6px; margin-bottom: 15px; border-left: 4px solid #2196F3;">
    <strong>📊 Parameter Ranges:</strong><br>
    • <strong>Max Fee:</strong> $1 - $5,000 (step: $10)<br>
    • <strong>Half Amount:</strong> $100 - $100,000 (step: $1,000)<br>
    • <strong>Key Amounts:</strong> $100, $1K, $10K, $100K (highlighted with colored markers)<br><br>
    <em>💡 Tip: All three curves intersect at (halfAmount, maxFee/2)</em>
</div>
"""))

display(controls, output)

## 📊 Comprehensive Analysis View

Generate a detailed 4-panel analysis with customizable parameters.

In [ ]:
# Comprehensive visualization with custom parameters
# Adjust these values as needed
MAX_FEE = 1000  # In tokens or dollars
HALF_AMOUNT = 10000
MAX_AMOUNT = 50000
USE_DOLLARS = False  # Set to True to display in USD

fig = create_comprehensive_visualization(
    max_fee=MAX_FEE,
    half_amount=HALF_AMOUNT,
    max_amount=MAX_AMOUNT,
    use_dollars=USE_DOLLARS
)
plt.show()

## 💾 Export Visualization

Export the comprehensive visualization as a single-line SVG file.

In [ ]:
# Export SVG with custom parameters
export_svg(
    max_fee=1000,
    half_amount=10000,
    max_amount=50000,
    use_dollars=False,
    filename='fee_curves.svg'
)

## 📊 Formula Reference

### Linear Fee
```
fee = min(maxFee, (amount × maxFee) / (2 × halfAmount))
```
- Simple linear growth until reaching cap at 2× halfAmount
- Most predictable and straightforward
- Constant percentage until cap is reached

### Progressive Fee
```
fee = (maxFee × amount²) / (halfAmount² + amount²)
```
- Fee percentage increases up to halfAmount, then decreases
- Encourages mid-sized transfers
- Smooth asymptotic approach to maxFee

### Regressive Fee
```
fee = (maxFee × amount) / (halfAmount + amount)
```
- Fee percentage continuously decreases as amount increases
- Most favorable for large transfers ("whale-friendly")
- Smooth asymptotic approach to maxFee

---

**Key Insight:** All three curves intersect at `(halfAmount, maxFee/2)` ✨

**Key Amounts:** The visualizations highlight 4 key transfer amounts ($100, $1K, $10K, $100K) with colored markers to help compare fee structures across common transaction sizes.

**USDC Conversion:** For USDC tokens, 1 USD = 10^6 tokens